In [114]:
import os
from datetime import datetime
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langgraph.checkpoint.memory import MemorySaver
import math
import numexpr

In [ ]:

# --- Configuración del LLM (Modelo de Lenguaje Grande) ---
# Inicializa el LLM con Ollama.
# Asegúrate de que Ollama esté corriendo y el modelo 'esté descargado.
try:
    llm = ChatOllama(model="llama3.2")
    print(f"Ollama inicializado con el modelo: {llm.model}")
except Exception as e:
    print(f"Error al inicializar Ollama: {e}")
    print("Asegúrate de que Ollama esté corriendo y el modelo especificado esté descargado.")
    exit("No se pudo inicializar el LLM de Ollama. Saliendo.")


# --- Definición de Herramientas ---

# 1. Herramienta de Calculadora
# Utiliza LLMMathChain para cálculos matemáticos.
# Requiere un LLM para procesar las expresiones.
@tool
def calculator_tool(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

# 2. Herramienta de busqueda en youtube
youtube_search_tool = YouTubeSearchTool()

# 3. Herramienta para obtener la hora actual
@tool
def get_current_time() -> str:
    """
    Devuelve la hora actual en un formato %Y-%m-%d %H:%M:%S
    """
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

get_current_time.name = "CurrentTime"
get_current_time.description = "Útil para obtener la hora y/o fecha actual. No necesita entrada."


# 4. Herramienta de Wikipedia
try:
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang='es'))
    wikipedia_tool = wikipedia
    wikipedia_tool.name = "Wikipedia"
    wikipedia_tool.description = "Útil para buscar información en Wikipedia. Recibe una consulta de búsqueda como entrada."
except Exception as e:
    print(f"Advertencia: No se pudo inicializar Wikipedia. Usando una herramienta de Wikipedia simulada. Error: {e}")
    @tool
    def wikipedia_tool(query: str) -> str:
        """Útil para buscar información en Wikipedia. Recibe una consulta de búsqueda como entrada."""
        print(f"Simulando búsqueda en Wikipedia para: '{query}'")
        if "inteligencia artificial" in query.lower():
            return "La inteligencia artificial (IA) es el campo de la ciencia de la computación dedicado a resolver problemas cognitivos comúnmente asociados con la inteligencia humana, como el aprendizaje, la resolución de problemas y el reconocimiento de patrones."
        else:
            return f"No se encontró información para '{query}' en Wikipedia simulada."


# Lista de todas las herramientas disponibles para el agente
tools = [calculator_tool, get_current_time, wikipedia_tool, youtube_search_tool]

# --- Creación del Agente ---

# Crea el agente
memory = MemorySaver()
agent_executor = create_react_agent(model=llm, tools=tools, prompt="""
Realiza el análisis adecuado para ayudar al usuario con su requerimiento. Siempre utiliza la información otorgada por las herramientas para generar tu respuesta."""
                                    , checkpointer=memory)

Ollama inicializado con el modelo: deepseek-r1


d:\taller_gen_ai\.venv\Lib\site-packages\langchain_core\tools\base.py:1314: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  fields = getattr(cls, "model_fields", {}) or getattr(cls, "__fields__", {})
d:\taller_gen_ai\.venv\Lib\site-packages\langchain_core\tools\base.py:1314: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  fields = getattr(cls, "model_fields", {}) or getattr(cls, "__fields__", {})
d:\taller_gen_ai\.venv\Lib\site-packages\langchain_core\tools\base.py:1314: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V

In [116]:
config = {"configurable": {"thread_id": "1"}}

In [117]:
# --- Ejemplos de Uso ---
print("\n--- Demostración del Agente ---")

# Ejemplo 1: Usando la calculadora
print("\nPregunta 1: ¿Cuánto es 12345 * 6789?")
response1 = agent_executor.invoke(
    {"messages": [{"role": "user", "content": "¿Cuánto es 12345 * 6789?"}]},
    config,
)
for m in response1["messages"]:
    m.pretty_print()


--- Demostración del Agente ---

Pregunta 1: ¿Cuánto es 12345 * 6789?


ResponseError: registry.ollama.ai/library/deepseek-r1:latest does not support tools (status code: 400)

In [110]:
# Ejemplo 2: Usando la búsqueda web
print("\nPregunta 2: ¿Cuál es la capital de España?")
response2 = agent_executor.invoke(
    {"messages": [{"role": "user", "content": "¿Cuál es la capital de España?"}]},
    config,
)
for m in response2["messages"]:
    m.pretty_print()


Pregunta 2: ¿Cuál es la capital de España?
================================ Human Message =================================

Busca en youtube y dame una lista de videos y urls relacionados con el Ecuador
================================== Ai Message ==================================
Tool Calls:
  youtube_search (1539924f-a17e-43c8-af71-f18c60860ec7)
 Call ID: 1539924f-a17e-43c8-af71-f18c60860ec7
  Args:
    query: Ecuador,10
================================= Tool Message =================================
Name: youtube_search

['https://www.youtube.com/watch?v=1eeLYNIliBY&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/watch?v=9cQlVww0zKo&list=RD9cQlVww0zKo&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/watch?v=H8wwRdIMUkQ&list=RDH8wwRdIMUkQ&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/shorts/RmSsTGclnzM', 'https://www.youtube.com/watch?v=YLK_HxMbcZ4&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/watch?v=Gx-hDQzbxfk&pp=ygUHRWN1YWRvc

In [111]:
# Ejemplo 2: Usando la herramienta de tiempo
print("\nPregunta 2: ¿Qué hora es ahora?")
response3 = agent_executor.invoke(
    {"messages": [{"role": "user", "content": "¿Qué dia es hoy?"}]}, 
    config,
)
for m in response3["messages"]:
    m.pretty_print()


Pregunta 2: ¿Qué hora es ahora?
================================ Human Message =================================

Busca en youtube y dame una lista de videos y urls relacionados con el Ecuador
================================== Ai Message ==================================
Tool Calls:
  youtube_search (1539924f-a17e-43c8-af71-f18c60860ec7)
 Call ID: 1539924f-a17e-43c8-af71-f18c60860ec7
  Args:
    query: Ecuador,10
================================= Tool Message =================================
Name: youtube_search

['https://www.youtube.com/watch?v=1eeLYNIliBY&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/watch?v=9cQlVww0zKo&list=RD9cQlVww0zKo&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/watch?v=H8wwRdIMUkQ&list=RDH8wwRdIMUkQ&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/shorts/RmSsTGclnzM', 'https://www.youtube.com/watch?v=YLK_HxMbcZ4&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/watch?v=Gx-hDQzbxfk&pp=ygUHRWN1YWRvcg%3D%3D', '

In [67]:
# Ejemplo 3: Combinando herramientas (si el LLM es lo suficientemente inteligente para encadenar)
print("\nPregunta 4: Busca en Wikipedia sobre 'Cambio Climático' y luego dime la hora actual.")
response4 = agent_executor.invoke({"messages": [{"role": "user", "content": "Busca en Wikipedia sobre 'Cambio Climático', dame esta información y también dime la hora actual."}]})
for m in response4['messages']:
    m.pretty_print()


Pregunta 4: Busca en Wikipedia sobre 'Cambio Climático' y luego dime la hora actual.
================================ Human Message =================================

Busca en Wikipedia sobre 'Cambio Climático', dame esta información y también dime la hora actual.
================================== Ai Message ==================================

", "parameters": {}}
Tool Calls:
  Wikipedia (498e0a53-6200-4f0e-aab3-59faef789b1c)
 Call ID: 498e0a53-6200-4f0e-aab3-59faef789b1c
  Args:
    query: Cambio Climático
  CurrentTime (75983599-4b63-4e20-988b-ffe1604782d3)
 Call ID: 75983599-4b63-4e20-988b-ffe1604782d3
  Args:
================================= Tool Message =================================
Name: Wikipedia

Page: Cambio climático
Summary: Un cambio climático se define[1]​[2]​ como la variación en el estado del sistema climático terrestre, formado por la atmósfera, la hidrosfera, la criosfera, la litosfera y la biosfera, que perdura durante periodos de tiempo suficientemente largos 

In [112]:
# Ejemplo 5: Pregunta que requiere búsqueda y posible cálculo (si el LLM lo interpreta)
print(
    "\nPregunta 5: ¿Cuál es la población actual de China y cuánto sería si aumentara en un 10%?"
)
response5 = agent_executor.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "¿Cuál es la población actual de China y cuánto sería si aumentara en un 10%?",
            }
        ]
    },
    config,
)
for m in response5["messages"]:
    m.pretty_print()


Pregunta 5: ¿Cuál es la población actual de China y cuánto sería si aumentara en un 10%?
================================ Human Message =================================

Busca en youtube y dame una lista de videos y urls relacionados con el Ecuador
================================== Ai Message ==================================
Tool Calls:
  youtube_search (1539924f-a17e-43c8-af71-f18c60860ec7)
 Call ID: 1539924f-a17e-43c8-af71-f18c60860ec7
  Args:
    query: Ecuador,10
================================= Tool Message =================================
Name: youtube_search

['https://www.youtube.com/watch?v=1eeLYNIliBY&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/watch?v=9cQlVww0zKo&list=RD9cQlVww0zKo&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/watch?v=H8wwRdIMUkQ&list=RDH8wwRdIMUkQ&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/shorts/RmSsTGclnzM', 'https://www.youtube.com/watch?v=YLK_HxMbcZ4&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.yo

In [113]:
# Ejemplo 6: Pregunta sobre vides de espol en youtube
print("\nPregunta 5: Dame una lista de videos relacionados con el Ecuador en youtube")

response5 = agent_executor.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Busca en youtube y dame una lista de videos y urls relacionados con el Ecuador",
            }
        ],
    },
    config,
)
for m in response5["messages"]:
    m.pretty_print()


Pregunta 5: Dame una lista de videos relacionados con el Ecuador en youtube
================================ Human Message =================================

Busca en youtube y dame una lista de videos y urls relacionados con el Ecuador
================================== Ai Message ==================================
Tool Calls:
  youtube_search (1539924f-a17e-43c8-af71-f18c60860ec7)
 Call ID: 1539924f-a17e-43c8-af71-f18c60860ec7
  Args:
    query: Ecuador,10
================================= Tool Message =================================
Name: youtube_search

['https://www.youtube.com/watch?v=1eeLYNIliBY&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/watch?v=9cQlVww0zKo&list=RD9cQlVww0zKo&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/watch?v=H8wwRdIMUkQ&list=RDH8wwRdIMUkQ&start_radio=1&pp=ygUHRWN1YWRvcqAHAQ%3D%3D', 'https://www.youtube.com/shorts/RmSsTGclnzM', 'https://www.youtube.com/watch?v=YLK_HxMbcZ4&pp=ygUHRWN1YWRvcg%3D%3D', 'https://www.youtube.com/wat